In [1]:
from bs4 import BeautifulSoup
import requests
import re
import fractions

In [2]:
"""

web scraper

"""

def get_numbers(lst):
    new_list = [int(x.split(':')[1]) if ':' in x else -1 for x in lst]
    return new_list

class RecipeFetcher:

    search_base_url = 'https://www.allrecipes.com/search/results/?wt=%s&sort=re'

    def search_recipes(self, keywords): 
        search_url = self.search_base_url %(keywords.replace(' ','+'))

        page_html = requests.get(search_url)
        page_graph = BeautifulSoup(page_html.content)

        return [recipe.a['href'] for recipe in\
               page_graph.find_all('div', {'class':'grid-card-image-container'})]

    def scrape_recipe(self, recipe_url):
        results = {}

        page_html = requests.get(recipe_url)
        print(recipe_url)
        page_graph = BeautifulSoup(page_html.content)
        
        find_name = [name.text for name in\
                          page_graph.find_all('h1', {'id': 'recipe-main-content'}, {'class': 'recipe-summary__h1'})]
        results['name'] = find_name[0]

        results['ingredients'] = [ingredient.text for ingredient in\
                                  page_graph.find_all('span', {'itemprop':'recipeIngredient'})]

        results['directions'] = [direction.text.strip() for direction in\
                                 page_graph.find_all('span', {'class':'recipe-directions__list--item'})
                                 if direction.text.strip()]

        results['nutrition'] = self.scrape_nutrition_facts(recipe_url)
        
        results['calories_and_servings'] = self.scrape_calories_servings(recipe_url)

        return results
    
    def scrape_nutrition_facts(self, recipe_url):
        results = []

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for nutrient_row in page_graph.find_all('div', {'class': 'nutrition-row'}):
            
            lst = nutrient_row.text.split(':')
            amount_lst = lst[1]
            name = lst[0].replace('\n', '')
            
            amount = amount_lst.split('\n')
            amount = [x.replace(' ', '') for x in amount[:2]]
            
            nutrient[name] = amount
            
        return nutrient
    
    def scrape_calories_servings(self, recipe_url):
        """
        returns [servings per recipe, amt per serving, calories]
        """

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for row in page_graph.find_all('div', {'class': 'nutrition-top light-underline'}):
            lst = row.text.split('\n')
            lst = list(filter(lambda a: a != '\r', lst))
            
            calories = [x.lstrip() for x in lst]
            calories.pop()
            info = get_numbers(calories)
            
            return info

In [60]:
rf = RecipeFetcher()
fluffy_pancakes = rf.search_recipes('meat lasagna')[0]
res = rf.scrape_recipe(fluffy_pancakes)

https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/


In [61]:
res

{'name': 'Classic and Simple Meat Lasagna',
 'ingredients': ['12 whole wheat lasagna noodles',
  '1 pound lean ground beef',
  '2 cloves garlic, chopped',
  '1/2 teaspoon garlic powder',
  '1 teaspoon dried oregano, or to taste',
  'salt and ground black pepper to taste',
  '1 (16 ounce) package cottage cheese',
  '2 eggs',
  '1/2 cup shredded Parmesan cheese',
  '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
  '2 cups shredded mozzarella cheese'],
 'directions': ['Preheat oven to 350 degrees F (175 degrees C).',
  'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.',
  'Place the ground beef into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the ski

In [62]:
res['name']

'Classic and Simple Meat Lasagna'

In [63]:
ingredients = res['ingredients']
ingredients

['12 whole wheat lasagna noodles',
 '1 pound lean ground beef',
 '2 cloves garlic, chopped',
 '1/2 teaspoon garlic powder',
 '1 teaspoon dried oregano, or to taste',
 'salt and ground black pepper to taste',
 '1 (16 ounce) package cottage cheese',
 '2 eggs',
 '1/2 cup shredded Parmesan cheese',
 '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
 '2 cups shredded mozzarella cheese']

In [64]:
def replace_veggie(meat):
    new_meat = meat.replace("(", "")
    new_meat = new_meat.replace(")", "")
    new_meat = new_meat.replace("/", " ")
    meat_lst = new_meat.split()
    
    original_quantity = []
    ingredient_name = ''

    # split ingredient into tokens to separate the 'quantity' from 'ingredient_name'
    for token in meat_lst:            
        if any([str(digit) in token for digit in range(10)]) and not any([char in token for char in ['(', ')']]):
            fraction_obj = sum(map(fractions.Fraction, token.split()))
            as_float = int(fraction_obj)
            original_quantity.append(as_float)
        else:
            ingredient_name = ingredient_name + ' ' + token
       

    new_quantity = original_quantity
    
    if len(new_quantity) > 1:
        num = new_quantity[-1]
    else:
        num = new_quantity[0]
      
    replace_idx = meat_lst.index(str(num)) + 2
    
    replace_term = meat_lst[replace_idx: ]
    s = " "
    term = s.join(replace_term)
    veggie = meat.replace(term, "chicken")
    return veggie   

In [65]:
def replace_meat(ingredients, meat, type_of_meat_lst, first):
    
    new_meat = meat.replace("(", "")
    new_meat = new_meat.replace(")", "")
    new_meat = new_meat.replace("/", " ")
    meat_lst = new_meat.split()
    
    original_quantity = []
    ingredient_name = ''

    # split ingredient into tokens to separate the 'quantity' from 'ingredient_name'
    for token in meat_lst:            
        if any([str(digit) in token for digit in range(10)]) and not any([char in token for char in ['(', ')']]):
            fraction_obj = sum(map(fractions.Fraction, token.split()))
            as_float = int(fraction_obj)
            original_quantity.append(as_float)
        else:
            ingredient_name = ingredient_name + ' ' + token
       

    new_quantity = original_quantity
    
    if len(new_quantity) > 1:
        num = new_quantity[-1]
    else:
        num = new_quantity[0]
      
    replace_idx = meat_lst.index(str(num)) + 2
    
    # second meat replacement
    if not first:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "veggie sausage")
        return veggie

    # ground meat replacement
    if type_of_meat_lst[0]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "ground tofu")
        return veggie
       
    # meat replacement
    if type_of_meat_lst[1]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "tofu")
        return veggie
        
    # sandwich meat replacement
    if type_of_meat_lst[2]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "impossible burger")
        return veggie
        
    # seafood replacement
    if type_of_meat_lst[3]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "tofuna fysh")
        return veggie
        
    # turkey/chicken sandwich meat replacement
    if type_of_meat_lst[4] or type_of_meat_lst[5]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "tofurkey")
        return veggie

In [66]:
def look_for_meat(ingredients):
    ground_meat_lst = ['ground beef', 'ground chicken', 'ground meat', 'ground turkey', 'ground lamb', 'ground pork', 'ground bison']
    meat_lst = ['chicken', 'steak', 'beef', 'lamb', 'bacon', 'pork', 'duck', 'bison', 'rabbit', 'cow', 'sausage', 'turkey']
    sandwich_meat_lst = ['hamburger', 'cheeseburger', 'sloppy joe']
    turkey_sandwich = ['turkey', 'slice']
    chicken_sandwich = ['chicken', 'slice']
    seafood_lst = ['salmon', 'cod', 'fish', 'halibut', 'shellfish', 'crab', 'lobster', 'shrimp', 'prawn', 'scallop']
    vegetarians_lst = ['tofu', 'tofurkey', 'impossible burger', 'veggie burger']
    
    ground = False
    meat = False
    sandwich = False
    seafood = False
    turkey = False
    chicken = False
    
    vegetarian = True
    vegetarian_recipe = True
    
    first = True

    for n, ingredient in enumerate(ingredients):
        if any(x in ingredient.lower() for x in vegetarians_lst):
            new_ingredient = replace_veggie(ingredient)
            ingredients[n] = new_ingredient
            vegetarian = False
            break
            
        if 'bread' not in ingredient.lower():
            #if any(x in ingredient.lower() for x in turkey_sandwich):
            if 'slice' in ingredient.lower() and 'turkey' in ingredient.lower():
                turkey = True
                replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
                new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
                ingredients[n] = new_ingredient
                first = False
                vegetarian = False
                vegetarian_recipe = False
                continue
            #if any(x in ingredient.lower() for x in chicken_sandwich):
            if 'slice' in ingredient.lower() and 'chicken' in ingredient.lower():
                chicken = True
                replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
                new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
                ingredients[n] = new_ingredient
                first = False
                vegetarian = False
                vegetarian_recipe = False
                continue
        if any(x in ingredient.lower() for x in ground_meat_lst):
            ground = True
            replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
            new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
            ingredients[n] = new_ingredient
            first = False
            vegetarian = False
            vegetarian_recipe = False
            continue
        if any(x in ingredient.lower() for x in meat_lst):
            meat = True
            replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
            new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
            ingredients[n] = new_ingredient
            first = False
            vegetarian = False
            vegetarian_recipe = False
            continue
        if any(x in ingredient.lower() for x in sandwich_meat_lst):
            sandwich = True
            replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
            new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
            ingredients[n] = new_ingredient
            first = False
            vegetarian = False
            vegetarian_recipe = False
            continue
        if any(x in ingredient.lower() for x in seafood_lst):
            seafood = True
            replace_meat_lst = [ground, meat, sandwich, seafood, turkey, chicken]
            new_ingredient = replace_meat(ingredients, ingredient, replace_meat_lst, first)
            ingredients[n] = new_ingredient
            first = False
            vegetarian = False
            vegetarian_recipe = False
            continue
            
    if vegetarian:
        new_ingredient = '1 pound chicken breast'
        ingredients.append(new_ingredient)
              
    return ingredients, vegetarian_recipe

In [67]:
ingredients

['12 whole wheat lasagna noodles',
 '1 pound lean ground beef',
 '2 cloves garlic, chopped',
 '1/2 teaspoon garlic powder',
 '1 teaspoon dried oregano, or to taste',
 'salt and ground black pepper to taste',
 '1 (16 ounce) package cottage cheese',
 '2 eggs',
 '1/2 cup shredded Parmesan cheese',
 '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
 '2 cups shredded mozzarella cheese']

In [68]:
p, recipe = look_for_meat(ingredients)
p

['12 whole wheat lasagna noodles',
 '1 pound ground tofu',
 '2 cloves garlic, chopped',
 '1/2 teaspoon garlic powder',
 '1 teaspoon dried oregano, or to taste',
 'salt and ground black pepper to taste',
 '1 (16 ounce) package cottage cheese',
 '2 eggs',
 '1/2 cup shredded Parmesan cheese',
 '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
 '2 cups shredded mozzarella cheese']

In [69]:
recipe

False

In [70]:
def change_recipe(res, change_to_meat):
    directions = res['directions']
    ingredients = res['ingredients']
    name = res['name']
    if change_to_meat:
        new_directions = change_directions_to_meat(directions, ingredients, name)
    else:
        new_directions = change_directions_to_veggie(directions, ingredients, name)
    return new_directions

In [71]:
directions = res['directions']
ingredients = res['ingredients']
directions

['Preheat oven to 350 degrees F (175 degrees C).',
 'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.',
 'Place the ground beef into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet. Cook the meat, chopping it into small chunks as it cooks, until no longer pink, about 10 minutes. Drain excess grease.',
 'In a bowl, mix the cottage cheese, eggs, and Parmesan cheese until thoroughly combined.',
 'Place 4 noodles side by side into the bottom of a 9x13-inch baking pan; top with a layer of the tomato-basil sauce, a layer of ground beef mixture, and a layer of the cottage cheese mixture. Repeat layers twice more, ending with a layer of sauce; sprin

In [72]:
ingredients

['12 whole wheat lasagna noodles',
 '1 pound ground tofu',
 '2 cloves garlic, chopped',
 '1/2 teaspoon garlic powder',
 '1 teaspoon dried oregano, or to taste',
 'salt and ground black pepper to taste',
 '1 (16 ounce) package cottage cheese',
 '2 eggs',
 '1/2 cup shredded Parmesan cheese',
 '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
 '2 cups shredded mozzarella cheese']

In [73]:
def get_veggie_burger(directions, ingredients):
    add_meat = True
    burger = ['pattie', 'burger', 'patti', 'hamburger', 'cheeseburger']
    new_dir = "Grill each side of veggie pattie for 5 minutes on grill."
    
    for n, step in enumerate(directions):
        if any(burger) in step.lower().split():
            directions[n] = new_dir
            
    if add_meat:
        directions.append(new_dir)
        
    return directions

In [74]:
def get_veggie_pasta(directions, ingredients):
    add_meat = True
    meat_lst = ['chicken', 'steak', 'beef', 'lamb', 'bacon', 'pork', 'duck', 'bison', 'rabbit', 'cow', 'sausage', 'turkey']
    new_dir = "Drizzle pan with olive oil. Stir fry chopped tofu in pan over medium heat until golden brown. Mix tofu with pasta."
    
    for n, step in enumerate(directions):
        if any(meat_lst) in step.lower().split():
            directions[n] = new_dir
            add_meat = False
            
    if add_meat:
        directions.append(new_dir)
        
    return directions

In [75]:
def get_veggie_rice(directions, ingredients):
    add_meat = True
    meat_lst = ['chicken', 'steak', 'beef', 'lamb', 'bacon', 'pork', 'duck', 'bison', 'rabbit', 'cow', 'sausage', 'turkey']
    new_dir = "Drizzle pan with olive oil. Stir fry chopped tofu in pan over medium heat until golden brown. Mix tofu with rice."
    
    for n, step in enumerate(directions):
        if any(meat_lst) in step.lower().split():
            directions[n] = new_dir
            add_meat = False
            
    if add_meat:
        directions.append(new_dir)
        
    return directions

In [76]:
def get_veggie_main(directions, ingredients):
    add_meat = True
    meat_lst = ['chicken', 'steak', 'beef', 'lamb', 'bacon', 'pork', 'duck', 'bison', 'rabbit', 'cow', 'sausage', 'turkey']
    new_dir = "Drizzle pan with olive oil. Stir fry chopped tofu in pan over medium heat until golden brown."
    
    for n, step in enumerate(directions):
        if any(meat_lst) in step.lower().split():
            directions[n] = new_dir
            add_meat = False
            
    if add_meat:
        directions.append(new_dir)
        
    return directions

In [77]:
def replace_veggie_direction(direction, old_word, new_word):
    words = direction.lower().split()
    
    idx = words.index(old_word)
    replace_this = words[idx+1]
    
    lower_dir = direction.lower()
    new_direction = lower_dir.replace(replace_this, new_word)
    
    return new_direction

In [78]:
def get_veggie_ground(directions, ingredients):
    print("getting ground veggie directions")
    for n, step in enumerate(directions):
        if 'ground' in step.lower().split():
            directions[n] = replace_veggie_direction(step, 'ground', 'tofu')  
    return directions

In [79]:
def change_directions_to_veggie(directions, ingredients, name):
    food_category = get_category_recipe(name)
    print(food_category)
    
    if food_category is 'pasta':
        new_directions = get_veggie_pasta(directions, ingredients)
        return new_directions
    elif food_category is 'burger':
        new_directions = get_veggie_burger(directions, ingredients)
        return new_directions
    elif food_category is 'main protein':
        new_directions = get_veggie_main(directions, ingredients)
        return new_directions
    elif food_category is 'ground':
        new_directions = get_veggie_ground(directions, ingredients)
        return new_directions
    elif food_category is 'rice':
        new_directions = get_veggie_rice(directions, ingredients)
        return new_directions
    else:
        new_directions = get_veggie_main(directions, ingredients)
        return new_directions

In [80]:
def replace_direction(direction, old_word, new_word):
    words = direction.lower()
    new_direction = words.replace(old_word, new_word)
    return new_direction

In [81]:
def get_meat_burger(directions, ingredients):
    add_meat = True
    burger = ['impossible', 'burger']
    
    for n, step in enumerate(directions):
        if 'impossible' in step.lower().split():
            directions[n] = "Form pattie shapes with hamburger meat. Grill each side of pattie for 5 minutes at medium heat."
            add_meat = False
        elif 'veggie' in step.lower().split():
            directions[n] = "Form pattie shapes with hamburger meat. Grill each side of pattie for 5 minutes at medium heat."
            add_meat = False
        elif 'patti' in step.lower().split():
            directions[n] = "Form pattie shapes with hamburger meat. Grill each side of pattie for 5 minutes at medium heat."
            add_meat = False   
            
    if add_meat:
        new_dir = "Form pattie shapes with hamburger meat. Grill each side of pattie for 5 minutes at medium heat."
        directions.append(new_dir)
        
    return directions

In [82]:
def get_meat_pasta(directions, ingredients):
    add_meat = True
    burger = ['impossible', 'burger']
    
    for n, step in enumerate(directions):
        if 'tofurkey' in step.lower().split():
            directions[n] = replace_direction(step, 'tofurkey', 'chicken')
            add_meat = False
        if 'tofu' in step.lower().split():
            directions[n] = replace_direction(step, 'tofu', 'chicken')
            add_meat = False
        if 'seitan' in step.lower().split():
            directions[n] = replace_direction(step, 'seitan', 'chicken')
            add_meat = False   
        if all(burger) in step.lower().split():
            directions[n] = replace_direction(step, 'burger', 'chicken')
            add_meat = False
            
    if add_meat:
        new_dir = "Add olive oil to a pan and chop chicken breast into bite size chunks. Then stir fry the chicken the pan at medium heat for 5 minutes each side. Mix chicken with pasta."
        directions.append(new_dir)
        
    return directions

In [83]:
def get_meat_main(directions, ingredients):
    add_meat = True
    
    for n, step in enumerate(directions):
        if 'tofurkey' in step.lower().split():
            directions[n] = replace_direction(step, 'tofurkey', 'chicken')
            add_meat = False
        if 'tofu' in step.lower().split():
            directions[n] = replace_direction(step, 'tofu', 'chicken')
            add_meat = False
        if 'seitan' in step.lower().split():
            directions[n] = replace_direction(step, 'seitan', 'chicken')
            add_meat = False   
            
    if add_meat:
        new_dir = "Make sure oven is at 400 degrees Farenheit. Place turkey breast on tinfoil and put in oven for 45 minutes."
        directions.append(new_dir)
        
    return directions

In [84]:
def get_meat_rice(directions, ingredients):
    add_meat = True
    burger = ['impossible', 'burger']
    
    for n, step in enumerate(directions):
        if 'tofurkey' in step.lower().split():
            directions[n] = replace_direction(step, 'tofurkey', 'chicken')
            add_meat = False
        if 'tofu' in step.lower().split():
            directions[n] = replace_direction(step, 'tofu', 'chicken')
            add_meat = False
        if 'seitan' in step.lower().split():
            directions[n] = replace_direction(step, 'seitan', 'chicken')
            add_meat = False   
        if all(burger) in step.lower().split():
            directions[n] = replace_direction(step, 'burger', 'chicken')
            add_meat = False
            
    if add_meat:
        new_dir = "Add olive oil to a pan and chop chicken breast into bite size chunks. Then stir fry the chicken the pan at medium heat for 5 minutes each side. Mix chicken with rice."
        directions.append(new_dir)
        
    return directions

In [85]:
def get_meat_ground(directions, ingredients):
    add_meat = True
    burger = ['impossible', 'burger']
    
    for n, step in enumerate(directions):
        if 'tofurkey' in step.lower().split():
            directions[n] = replace_direction(step, 'tofurkey', 'ground chicken')
            add_meat = False
        if 'tofu' in step.lower().split():
            directions[n] = replace_direction(step, 'tofu', 'ground chicken')
            add_meat = False
        if 'seitan' in step.lower().split():
            directions[n] = replace_direction(step, 'seitan', 'ground chicken')
            add_meat = False   
        if all(burger) in step.lower().split():
            directions[n] = replace_direction(step, 'burger', 'ground chicken')
            add_meat = False
            
    if add_meat:
        new_dir = "Grind chicken and mix with rest of ingredients."
        directions.append(new_dir)
        
    return directions

In [86]:
def change_directions_to_meat(directions, ingredients, name):
    food_category = get_category_recipe(name)
    print(food_category)
    
    if food_category is 'pasta':
        new_directions = get_meat_pasta(directions, ingredients)
        return new_directions
    elif food_category is 'burger':
        new_directions = get_meat_burger(directions, ingredients)
        return new_directions
    elif food_category is 'main protein':
        new_directions = get_meat_main(directions, ingredients)
        return new_directions
    elif food_category is 'ground':
        new_directions = get_meat_ground(directions, ingredients)
        return new_directions
    elif food_category is 'rice':
        new_directions = get_meat_rice(directions, ingredients)
        return new_directions
    else:
        new_directions = get_meat_main(directions, ingredients)
        return new_directions

In [87]:
def get_category_recipe(recipe_name):
    name = recipe_name.lower()
    pasta_lst = ['pasta', 'noodle', 'mein', 'pad thai']
    main_lst = ['roasted', 'fried', 'baked']

    print(name)
    
    if any(pasta_lst) in name.split():
        return 'pasta'
    if 'burger' in name:
        return 'burger'
    if  any(main_lst) in name.split():
        return 'main protein'
    if 'lasagna' in name:
        return 'ground'
    if 'rice' in name:
        return 'rice'
    return 'main protein'

In [88]:
res

{'name': 'Classic and Simple Meat Lasagna',
 'ingredients': ['12 whole wheat lasagna noodles',
  '1 pound ground tofu',
  '2 cloves garlic, chopped',
  '1/2 teaspoon garlic powder',
  '1 teaspoon dried oregano, or to taste',
  'salt and ground black pepper to taste',
  '1 (16 ounce) package cottage cheese',
  '2 eggs',
  '1/2 cup shredded Parmesan cheese',
  '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
  '2 cups shredded mozzarella cheese'],
 'directions': ['Preheat oven to 350 degrees F (175 degrees C).',
  'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.',
  'Place the ground beef into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet.

In [89]:
recipe

False

In [90]:
d = change_recipe(res, recipe)

classic and simple meat lasagna
ground
getting ground veggie directions


In [91]:
d

['Preheat oven to 350 degrees F (175 degrees C).',
 'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.',
 'place the ground tofu into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet. cook the meat, chopping it into small chunks as it cooks, until no longer pink, about 10 minutes. drain excess grease.',
 'In a bowl, mix the cottage cheese, eggs, and Parmesan cheese until thoroughly combined.',
 'place 4 noodles side by side into the bottom of a 9x13-inch baking pan; top with a layer of the tomato-basil sauce, a layer of ground tofu mixture, and a layer of the cottage cheese mixture. repeat layers twice more, ending with a layer of sauce; sprin

In [59]:
res

{'name': 'Classic and Simple Meat Lasagna',
 'ingredients': ['12 whole wheat lasagna noodles',
  '1 pound ground tofu',
  '2 cloves garlic, chopped',
  '1/2 teaspoon garlic powder',
  '1 teaspoon dried oregano, or to taste',
  'salt and ground black pepper to taste',
  '1 (16 ounce) package cottage cheese',
  '2 eggs',
  '1/2 cup shredded Parmesan cheese',
  '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
  '2 cups shredded mozzarella cheese'],
 'directions': ['Preheat oven to 350 degrees F (175 degrees C).',
  'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.',
  'place the ground tofu into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet.

In [165]:
h = change_directions_to_meat(directions, ingredients, res['name'])
h

pasta


['Preheat oven to 350 degrees F (175 degrees C). Prepare asparagus by trimming woody base and cutting diagonally into 4 inch pieces.',
 'In a roasting pan, combine asparagus, bell pepper, mushrooms, roasted garlic and tomato. Sprinkle with rosemary and oregano, then drizzle with olive oil. Bake in preheated oven for 15 minutes.',
 'Bring a large pot of lightly salted water to a boil. Add pasta and cook for 8 to 10 minutes or until al dente; drain. Toss with Parmesan cheese, tapenade and roasted vegetables.',
 'Add olive oil to a pan and chop chicken breast into bite size chunk. Then stir fry the chicken the pan at medium heat for 5 minutes each side.',
 'Add olive oil to a pan and chop chicken breast into bite size chunks. Then stir fry the chicken the pan at medium heat for 5 minutes each side.']

In [62]:
res

{'ingredients': ['1/4 pound fresh asparagus',
  '2 red bell pepper, sliced',
  '1/4 pound crimini mushrooms, sliced',
  '10 cloves roasted garlic, chopped',
  '1/2 tomato, quartered',
  '1/2 teaspoon chopped fresh rosemary',
  '1/2 teaspoon chopped fresh oregano',
  '2 tablespoons olive oil',
  '8 ounces dry fettuccini noodles',
  '1/4 cup grated Parmesan cheese',
  '2 tablespoons tapenade',
  '1 pound chicken breast'],
 'directions': ['Preheat oven to 350 degrees F (175 degrees C). Prepare asparagus by trimming woody base and cutting diagonally into 4 inch pieces.',
  'In a roasting pan, combine asparagus, bell pepper, mushrooms, roasted garlic and tomato. Sprinkle with rosemary and oregano, then drizzle with olive oil. Bake in preheated oven for 15 minutes.',
  'Bring a large pot of lightly salted water to a boil. Add pasta and cook for 8 to 10 minutes or until al dente; drain. Toss with Parmesan cheese, tapenade and roasted vegetables.'],
 'nutrition': {'Total Fat': ['14.6g', '23%']